In [1]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import models, transforms

import matplotlib.pyplot as plt
import time
import os

%load_ext autoreload
%autoreload 2

plt.ion()   # interactive mode

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, num_epochs=25):
    use_gpu = True
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda().float())
                    labels = Variable(labels.cuda().float())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs).view(-1)
                preds = outputs.data.cpu().numpy() > 0.5
                loss = criterion(outputs,labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss     += loss.cpu().data[0]
                running_corrects += np.equal(preds, labels.data.cpu().numpy()).mean()

            epoch_loss = running_loss / len(dataloader[phase])
            epoch_acc = running_corrects / len(dataloader[phase])

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
import h5py

means, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
means = np.array(means)[np.newaxis,:,np.newaxis,np.newaxis]
std   = np.array(std)[np.newaxis,:,np.newaxis,np.newaxis]

data_dir = "mitos-p300-preliminary.hdf5"
with h5py.File(data_dir, 'r') as ds:
    X  = (ds['X'][...].transpose((0,3,1,2)) - means) / std
    y  = ds['y'][...].astype("float32")
    domain = ds['domain'][...]

In [ ]:
train_data = torch.utils.data.TensorDataset(torch.from_numpy(X[domain!=3].astype('float32')),
                                            torch.from_numpy(y[domain!=3].astype('float32')))
val_data = torch.utils.data.TensorDataset(torch.from_numpy(X[domain==3].astype('float32')),
                                          torch.from_numpy(y[domain==3].astype('float32')))
dataloader = { 'train' : torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True, num_workers=8),
             'val' :torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=True, num_workers=8)}
use_gpu = torch.cuda.is_available()

print("done")

In [ ]:
class GaussianNoise(nn.Module):
    def __init__(self, stddev=0.05):
        super().__init__()
        self.stddev = stddev

    def forward(self, din):
        if self.training:
            return din + torch.autograd.Variable(torch.randn(din.size()).cuda() * self.stddev)
        return din

resnet = models.resnet34(pretrained=True)
resnet.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid())

model_ft = nn.Sequential(
    GaussianNoise(),
    resnet,
)
model_ft = model_ft.cuda()

criterion = nn.BCELoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, dataloader)

In [ ]:
model_ft.train(False)
p = []
t = []
for data in dataloader['val']:
    # get the inputs
    inputs, labels = data

    # wrap them in Variable
    if use_gpu:
        inputs = Variable(inputs.cuda().float())
        labels = Variable(labels.cuda().float())
    else:
        inputs, labels = Variable(inputs), Variable(labels)

    # forward
    outputs = model_ft(inputs).view(-1)
    p.append(outputs.data.cpu().numpy())
    t.append(labels.data.cpu().numpy())
p = np.concatenate(p, axis=0)
t = np.concatenate(t, axis=0)

In [1]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(t, p>.5))
confusion_matrix(t,p > 0.5)

NameError: name 't' is not defined

In [3]:
import h5py

means, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
means = np.array(means)[np.newaxis,:,np.newaxis,np.newaxis]
std   = np.array(std)[np.newaxis,:,np.newaxis,np.newaxis]

with h5py.File('mitos-p64-preliminary.hdf5') as ds:
    X = (ds['X'][...].transpose((0,3,1,2)) - means) / std
    y = ds['y'][...]
    domain = ds['domain'][...]
    
train_data = torch.utils.data.TensorDataset(torch.from_numpy(X[domain!=3].astype('float32')),
                                            torch.from_numpy(y[domain!=3].astype('float32')))
val_data = torch.utils.data.TensorDataset(torch.from_numpy(X[domain==3].astype('float32')),
                                          torch.from_numpy(y[domain==3].astype('float32')))
dataloader = { 'train' : torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True, num_workers=8),
             'val' :torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=True, num_workers=8)}
use_gpu = torch.cuda.is_available()

print("done")

done


In [ ]:
%matplotlib inline
plt.figure(figsize=(20,5))
plt.imshow(np.concatenate(X[np.random.choice(np.where(y==1)[0], size=(20,))], axis=1))
plt.show()
plt.figure(figsize=(20,5))
plt.imshow(np.concatenate(X[np.random.choice(np.where(y==0)[0], size=(20,))], axis=1))
plt.show()

In [7]:
'''ResNet in PyTorch.
For Pre-activation ResNet, see 'preact_resnet.py'.
Reference:
[1] Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun
    Deep Residual Learning for Image Recognition. arXiv:1512.03385
'''
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4*2, ceil_mode=True)
        out = out.view(out.size(0), -1)
        out = self.linear(out).view(out.size(0))
        return out


def ResNet18(*args,**kwargs):
    return ResNet(BasicBlock, [2,2,2,2],*args,**kwargs)

def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3],*args,**kwargs)

def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3],*args,**kwargs)

def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3],*args,**kwargs)

def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3],*args,**kwargs)

In [8]:
resnet = ResNet18(num_classes=1)
resnet = resnet.cuda()

criterion = nn.BCELoss()

optimizer_ft = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1503968623488/work/torch/lib/THC/generic/THCTensorCopy.c:18

In [6]:
train_model(resnet, criterion, optimizer_ft, exp_lr_scheduler, dataloader)

Epoch 0/24
----------


RuntimeError: cudaEventSynchronize in future::wait: device-side assert triggered

In [ ]:
model_ft.train(False)
model_ft.cuda()

fs = []
for i in range(0,len(X),10):
    Xb = (X[i:min(len(X),i+10)] - means) / std
    Xb = Xb.repeat(4,axis=2).repeat(4,axis=3)
    Xb = Variable(torch.from_numpy(Xb.astype('float32'))).cuda()
    f = model_ft(Xb)
    d = f.cpu().data.numpy()
    fs.append( d )
    print("{}/{}".format(i,len(X),end='\r'))
fs = np.concatenate(fs, axis=0)
fs.shape

In [ ]:
fs.shape

In [ ]:
import sklearn.manifold

tsne = sklearn.manifold.TSNE()

embed = tsne.fit_transform(fs)

In [ ]:
%matplotlib inline

plt.figure(figsize=(10,10))
plt.title("ResNet-34 Feature embedding. Color denotes Domain.")
plt.scatter(embed[y==0,0],embed[y==0,1],c=domain[y==0],marker='x',label="No Mitosis")
plt.scatter(embed[y==1,0],embed[y==1,1],c=domain[y==1],marker='+',label="Mitosis")
plt.legend()


In [ ]:
for d in np.unique(domain):